### RAG Pipeline - Data Ingestion to Vector DB Pipeline

In [8]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [9]:
### Read all the PDFs inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: 4 Types of LLM Architectueres.pdf
  ✓ Loaded 13 pages

Processing: AI AudioBook Generator.pdf
  ✓ Loaded 3 pages

Processing: CollegePeCharcha_VITPune.pdf
  ✓ Loaded 10 pages

Total documents loaded: 26


In [10]:
all_pdf_documents

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-17T04:52:00+00:00', 'title': 'Types of LLM Archs', 'moddate': '2025-08-17T04:51:57+00:00', 'keywords': 'DAGwQeT0M8w,BAGQ0G2eswY,0', 'author': 'content team', 'source': '..\\data\\pdf\\4 Types of LLM Architectueres.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1', 'source_file': '4 Types of LLM Architectueres.pdf', 'file_type': 'pdf'}, page_content='T y p e s  o f  L L M\nA r c h i t e c t u r e s\nBhavishya Pandit'),
 Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-17T04:52:00+00:00', 'title': 'Types of LLM Archs', 'moddate': '2025-08-17T04:51:57+00:00', 'keywords': 'DAGwQeT0M8w,BAGQ0G2eswY,0', 'author': 'content team', 'source': '..\\data\\pdf\\4 Types of LLM Architectueres.pdf', 'total_pages': 13, 'page': 1, 'page_label': '2', 'source_file': '4 Types of LLM Architectueres.pdf', 'file_type': 'pdf'}, page_content='Bhavishya Pandit\nIntroduction\nTransformers a

In [13]:
### Text splitting into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    if split_docs:
        print(f"Example chunk: ")
        print(f"Content: {split_docs[0].page_content[:200]}...\n")
        print(f"Metadata: {split_docs[0].metadata}\n")
    
    return split_docs

In [15]:
chunks = split_documents(all_pdf_documents)
chunks

Split 26 documents into 29 chunks
Example chunk: 
Content: T y p e s  o f  L L M
A r c h i t e c t u r e s
Bhavishya Pandit...

Metadata: {'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-17T04:52:00+00:00', 'title': 'Types of LLM Archs', 'moddate': '2025-08-17T04:51:57+00:00', 'keywords': 'DAGwQeT0M8w,BAGQ0G2eswY,0', 'author': 'content team', 'source': '..\\data\\pdf\\4 Types of LLM Architectueres.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1', 'source_file': '4 Types of LLM Architectueres.pdf', 'file_type': 'pdf'}



[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-17T04:52:00+00:00', 'title': 'Types of LLM Archs', 'moddate': '2025-08-17T04:51:57+00:00', 'keywords': 'DAGwQeT0M8w,BAGQ0G2eswY,0', 'author': 'content team', 'source': '..\\data\\pdf\\4 Types of LLM Architectueres.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1', 'source_file': '4 Types of LLM Architectueres.pdf', 'file_type': 'pdf'}, page_content='T y p e s  o f  L L M\nA r c h i t e c t u r e s\nBhavishya Pandit'),
 Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-17T04:52:00+00:00', 'title': 'Types of LLM Archs', 'moddate': '2025-08-17T04:51:57+00:00', 'keywords': 'DAGwQeT0M8w,BAGQ0G2eswY,0', 'author': 'content team', 'source': '..\\data\\pdf\\4 Types of LLM Architectueres.pdf', 'total_pages': 13, 'page': 1, 'page_label': '2', 'source_file': '4 Types of LLM Architectueres.pdf', 'file_type': 'pdf'}, page_content='Bhavishya Pandit\nIntroduction\nTransformers a

### embeddings and vectorStoreDB

In [17]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [19]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [23]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

# Generate  embeddings
embeddings = embedding_manager.generate_embeddings(texts)

#store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 29 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.50it/s]


Generated embeddings with shape: (29, 384)
Adding 29 documents to vector store...
Successfully added 29 documents to vector store
Total documents in collection: 29


### Retriever Pipeline from VectorStore

In [24]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever = RAGRetriever(vectorstore,embedding_manager)

In [26]:
rag_retriever.retrieve("How do transformers work?")

Retrieving documents for query: 'How do transformers work?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.32it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_f51cd3aa_2',
  'content': 'Bhavishya Pandit\nSource\nInstead of passing information step by step like RNNs or LSTMs,\ntransfomers introduced self-attention and feed-forward layers so\ntokens could look at each other all at once. That introduced te\nconcept of parallelism.\nTraining on massive datasets without crawling speeds\nScaling models from millions to billions of parameters\n1. Transformers\nThink of Transformers as the moment deep learning hit turbo\nmode.\nThis shift unlocked two big things:',
  'metadata': {'content_length': 466,
   'title': 'Types of LLM Archs',
   'keywords': 'DAGwQeT0M8w,BAGQ0G2eswY,0',
   'source_file': '4 Types of LLM Architectueres.pdf',
   'source': '..\\data\\pdf\\4 Types of LLM Architectueres.pdf',
   'total_pages': 13,
   'creationdate': '2025-08-17T04:52:00+00:00',
   'author': 'content team',
   'page_label': '3',
   'file_type': 'pdf',
   'page': 2,
   'producer': 'Canva',
   'creator': 'Canva',
   'doc_index': 2,
   'moddate': '2025-

### Integration of VectorDB Context Pipeline with LLM output

In [ ]:
##Simple RAG Pipeline integrating VectorDB context with LLM output
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize Gemini Chat LLM
gemini_api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    google_api_key=gemini_api_key,
    model="gemini-2.5-flash",
    temperature=0.1,
    max_tokens=1024
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k = 3):
    ## retrieve context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else "No relevant context found."

    ## generate response
    prompt = f"""Use the following context to answer the question:\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""

    response = llm.invoke([prompt.format(context = context, query = query)])
    return response.content

In [34]:
answer = rag_simple("Explain the transformer architecture.", rag_retriever, llm)
print(answer)

Retrieving documents for query: 'Explain the transformer architecture.'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.71it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


The transformer architecture, introduced in 2017, is considered the foundation of modern AI models and the groundwork for nearly every major language model today. Unlike RNNs or LSTMs that pass information step by step, transformers introduced self-attention and feed-forward layers. This allows tokens to look at each other all at once, which introduced the concept of parallelism. This shift unlocked the ability to train on massive datasets without crawling speeds and scale models from millions to billions of parameters.


### Enhanced RAG Pipeline

In [37]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is Audiobook Generator?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What is Audiobook Generator?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.42it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


Answer: AudioBook Generator is a web application that allows users to upload text documents (PDF, DOCX, TXT) and automatically converts them into high-quality audiobooks.
Sources: [{'source': 'AI AudioBook Generator.pdf', 'page': 0, 'score': 0.3885383605957031, 'preview': 'AudioBook Generator \n1. Introduction / Objective \nAudioBook Generator is a web application that allows users to upload one or more text \ndocuments (PDF, DOCX, TXT) and automatically converts them into high-quality \naudiobooks. The application leverages Large Language Models (LLMs) to rewrite extract...'}]
Confidence: 0.3885383605957031
Context Preview: AudioBook Generator 
1. Introduction / Objective 
AudioBook Generator is a web application that allows users to upload one or more text 
documents (PDF, DOCX, TXT) and automatically converts them into high-quality 
audiobooks. The application leverages Large Language Models (LLMs) to rewrite extract


In [39]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer, 
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Explain college reality revealed only after admission", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'Explain college reality revealed only after admission'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.70it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
COLLEGE PE
चच 
P r o b l e m s  :
Even after scoring well, there
is a mismatch between
expe

ctations and reality.
  Cutoffs don’t reflect academic
pressure, culture, placement focus,
or flexibility for other goals (like
startups or higher studies).
80% of students don’t get to know their
college until they arrive for admission
leads to uninformed decision due to lack
of real-time knowledge about the college
 60%+ depend on counsellors charging
₹ 3,000– ₹ 20,000  but 80% report
dissatisfaction, as suggestions are generic
and data-driven, not experience-driven.
 No reliable platform today
provides honest, branch-specific
college insights.
 High Scorers, Average
College Reality
P ast y ear cut of f s ≠ the full pictur e
 College Reality Revealed 
Only After Admission
 Expensive Counselling, Poor
Student Satisfaction
No Reliable Source for
College Truth
P R O B L E M
Business Overview

Question: Explain college reality revealed only after admission

Answer:

Final Answer: 80% of students don't get to know their college until they arrive for admission, leading to uninformed decisi